In [36]:
!pip install -q segmentation-models-pytorch albumentations==1.3.0


# Подключение к датасету

In [37]:
import torch
import segmentation_models_pytorch as smp
from torch.utils.data import DataLoader
from torch import nn
import os
import glob
from google.colab import drive
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset
# 📌 Устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 📁 Путь к тренировочным изображениям
train_dir = '/content/smokedata/Training/Training'

# ✅ Размер
image_size = (256, 256)

# 🧩 Трансформации для изображений
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor()
])

# Обучение модели сверточной

In [38]:
class SmokeDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_paths = sorted(glob.glob(os.path.join(image_dir, '*.jpg')))  # Это инициализация атрибута image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        # ⚠️ Генерируем ПСЕВДОмаску
        label = random.randint(0, 1)
        mask = torch.ones((1, image_size[0], image_size[1])) * label  # Маска
        mask = mask.unsqueeze(0)  # Добавляем batch размер: [1, 1, H, W]

        return image, mask

In [39]:
import os
import zipfile
import random
import shutil
drive.mount('/content/drive')

# 📌 Пути к архиву и извлечённой папке
zip_path = '/content/drive/MyDrive/Colab Notebooks/AiDatasets/smokedata.zip'
extract_dir = '/content/smokedata'

# 📌 Распаковываем архив
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

for root, dirs, files in os.walk(extract_dir):
  print(root, "содержит", len(files), "файлов")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/smokedata содержит 0 файлов
/content/smokedata/Testing содержит 0 файлов
/content/smokedata/Testing/Testing содержит 224 файлов
/content/smokedata/Validation содержит 0 файлов
/content/smokedata/Validation/Validation содержит 180 файлов
/content/smokedata/Training содержит 0 файлов
/content/smokedata/Training/Training содержит 716 файлов


In [40]:
# 📤 DataLoader
train_dataset = SmokeDataset(train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# 🧠 Модель
model = smp.Unet(encoder_name="resnet18", encoder_weights="imagenet", in_channels=3, classes=1, activation=None)
model.to(device)

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [45]:
# Путь к данным
train_dir = '/content/smokedata/Training/Training'

# Загрузка данных
train_dataset = SmokeDataset(train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)



In [43]:
# ⚙️ Loss и оптимизатор
loss_fn = smp.losses.DiceLoss(mode='binary')
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)


In [46]:
epochs = 3
model.train()
for epoch in range(epochs):
    total_loss = 0
    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

Epoch 1/3, Loss: 70.3790
Epoch 2/3, Loss: 62.5593
Epoch 3/3, Loss: 58.4423


In [76]:
# Пример вычисления метрик
outputs = model(images)
# Преобразуем в бинарные маски
predictions = (outputs > 0.5).float()  # Преобразуем предсказания в бинарные

# Преобразуем маски в тот же формат
masks = (masks > 0.5).float()

# Вычисляем IoU для каждого класса
iou_score = jaccard_score(masks.cpu().numpy().flatten(), predictions.cpu().numpy().flatten())
print(f"Jaccard Score (IoU): {iou_score}")

Jaccard Score (IoU): 0.2507892652275571


# Трансформерная

In [52]:

# Используем модель с энкодером 'mit_b0' (Vision Transformer от Microsoft)
modelTr = smp.Unet(
    encoder_name='mit_b0',  # Используем MIT (Mask Image Transformer) от Microsoft
    encoder_weights='imagenet',  # Загружаем веса с ImageNet
    in_channels=3,  # Каналы входного изображения
    classes=1,  # Один класс для сегментации (если бинарная сегментация)
)

# Оптимизатор и функция потерь

# Перемещаем модель на GPU, если доступен
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelTr = modelTr.to(device)  # Перемещаем модель на выбранное устройство

# Оптимизатор и функция потерь
optimizer = torch.optim.Adam(modelTr.parameters(), lr=0.0001)
loss_fn = smp.losses.DiceLoss(mode='binary')  # Для бинарной сегментации

In [53]:
# Цикл обучения
for epoch in range(epochs):
    modelTr.train()
    total_loss = 0
    for images, masks in train_loader:
        images = images.to(device)  # Перемещаем изображения на GPU
        masks = masks.to(device)  # Перемещаем маски на GPU

        optimizer.zero_grad()
        outputs = modelTr(images)  # Прогоняем данные через модель
        loss = loss_fn(outputs, masks)  # Вычисляем потери
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}")

Epoch 1/3, Loss: 0.4177185247064303
Epoch 2/3, Loss: 0.3704961834673109
Epoch 3/3, Loss: 0.36133601112738667


In [75]:
# Пример вычисления метрик
outputs = modelTr(images)
# Преобразуем в бинарные маски
predictions = (outputs > 0.5).float()  # Преобразуем предсказания в бинарные

# Преобразуем маски в тот же формат
masks = (masks > 0.5).float()

# Вычисляем IoU для каждого класса
iou_score = jaccard_score(masks.cpu().numpy().flatten(), predictions.cpu().numpy().flatten())
print(f"Jaccard Score (IoU): {iou_score}")

Jaccard Score (IoU): 0.24934069398205458


# Улучшение бейзлайна cnn
Гипотезы
Аугментация данных: Добавить больше разнообразия в обучающий набор данных с помощью различных техник аугментации:

Вращение, масштабирование, перевороты, изменение яркости и контраста.

Вставка случайного шума для повышения устойчивости модели.

Подбор моделей: Попробовать использовать более сложные архитектуры CNN (например, ResNet, DenseNet, EfficientNet, которые могут быть более эффективными для задач сегментации).

Подбор гиперпараметров:

Использовать более сложные функции потерь, например, комбинацию DiceLoss и CrossEntropyLoss для улучшения качества сегментации.

Эксперименты с learning rate, оптимизаторами (например, Adam, AdamW) и их параметрами.

Выбор более подходящего размера батча и числа эпох.

Предобученные веса: Использовать предобученные веса для более быстрых сходимости и лучшего качества модели.

In [69]:
import segmentation_models_pytorch as smp
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Пример улучшенной аугментации данных
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Случайная обрезка и изменение масштаба
    transforms.RandomVerticalFlip(),  # Дополнительная вертикальная симметрия
    transforms.ToTensor(),
])

# Используем модель ResNet-50 с предобученными весами
modelSVnew = smp.Unet(
    encoder_name="resnet50",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    activation=None  # Для использования DiceLoss с бинарной классификацией
)

In [70]:
# Настроим оптимизатор и функцию потерь
optimizer = optim.Adam(modelSVnew.parameters(), lr=1e-4)

# Используем DiceLoss напрямую из smp
loss_fn = smp.losses.DiceLoss(mode='binary')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = modelSVnew.to(device)

# Настроим scheduler для уменьшения скорости обучения, если модель не улучшает результат
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5)

In [71]:
# Обучение
for epoch in range(3):
    modelSVnew.train()
    total_loss = 0
    for images, masks in train_loader:
        images = images.to(device)  # Перемещаем изображения на GPU
        masks = masks.to(device)  # Перемещаем маски на GPU

        optimizer.zero_grad()
        outputs = modelSVnew(images)  # Проходим через модель
        loss = loss_fn(outputs, masks)  # Вычисляем потери
        loss.backward()  # Обратное распространение
        optimizer.step()  # Шаг оптимизации

        total_loss += loss.item()

    # Понижение скорости обучения, если потери не улучшаются
    scheduler.step(total_loss)

    print(f"Epoch [{epoch+1}/3], Loss: {total_loss/len(train_loader):.4f}")

Epoch [1/3], Loss: 0.4065
Epoch [2/3], Loss: 0.3600
Epoch [3/3], Loss: 0.3347


In [60]:
from sklearn.metrics import jaccard_score

# Пример вычисления метрик
outputs = modelSVnew(images)
# Преобразуем в бинарные маски
predictions = (outputs > 0.5).float()  # Преобразуем предсказания в бинарные

# Преобразуем маски в тот же формат
masks = (masks > 0.5).float()

# Вычисляем IoU для каждого класса
iou_score = jaccard_score(masks.cpu().numpy().flatten(), predictions.cpu().numpy().flatten())
print(f"Jaccard Score (IoU): {iou_score}")

Jaccard Score (IoU): 0.4899922532980725


# 2. Трансформерные модели (ViT, Swin Transformer) - улучшение бейзлайна
a. Гипотезы
Аугментация данных: Применить такие же методы аугментации, как и для сверточных моделей (например, случайные перевороты, изменения яркости).

Подбор моделей: Использование трансформеров, например, Vision Transformer (ViT) или Swin Transformer. Эти модели могут захватывать более сложные зависимости между пикселями за счет их внимания.

Подбор гиперпараметров:

Эксперименты с различными размерами патчей для ViT.

Использование более сложных обучающих техник, например, увеличение числа слоев внимания.

Предобученные веса: Использование предобученных весов для трансформеров (например, с ImageNet).

In [65]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Пример аугментации
transform = A.Compose([
    A.Resize(224, 224),
    A.RandomCrop(224, 224),
    A.HorizontalFlip(),
    A.RandomBrightnessContrast(),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# Обновим наш Dataset для использования аугментаций
class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, mask_paths, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        mask = Image.open(self.mask_paths[idx]).convert("L")

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask


In [66]:
# Используем learning rate scheduler для динамической коррекции
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


In [73]:
import segmentation_models_pytorch as smp
import torch.optim as optim

# Создаем модель с использованием ViT как энкодер
modelTRnew = smp.Unet(
    encoder_name="mit_b0",  # Использование ViT
    encoder_weights="imagenet",  # Предобученные веса
    in_channels=3,
    classes=1
)

# Настроим оптимизатор и функцию потерь
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_fn = smp.losses.DiceLoss(mode='binary')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Перемещаем модель на GPU
modelTRnew = modelTRnew.to(device)

# Обучение
for epoch in range(3):
    modelTRnew.train()
    total_loss = 0
    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()
        outputs = modelTRnew(images)
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    scheduler.step()
    print(f"Epoch [{epoch+1}/3], Loss: {total_loss/len(train_loader):.4f}")


Epoch [1/3], Loss: 0.4664
Epoch [2/3], Loss: 0.4810
Epoch [3/3], Loss: 0.4570


In [74]:
# Пример вычисления метрик
outputs = modelTRnew(images)
# Преобразуем в бинарные маски
predictions = (outputs > 0.5).float()  # Преобразуем предсказания в бинарные

# Преобразуем маски в тот же формат
masks = (masks > 0.5).float()

# Вычисляем IoU для каждого класса
iou_score = jaccard_score(masks.cpu().numpy().flatten(), predictions.cpu().numpy().flatten())
print(f"Jaccard Score (IoU): {iou_score}")

Jaccard Score (IoU): 0.16334943436600216


## Сравнение Jaccard Score (IoU) моделей сегментации

| Тип модели                | Версия        | Jaccard Score (IoU) |
|--------------------------|---------------|----------------------|
|  Сверточная (CNN)       | Дефолтная     | **0.2508**           |
|  Трансформерная (Transformer) | Дефолтная     | **0.2493**           |
|  Сверточная (CNN)       | Улучшенная    | **0.4900**         |
|  Трансформерная (Transformer) | Улучшенная    | **0.1633**         |

###  Выводы:
- Улучшенная **сверточная модель** показала **наилучший результат**.
- Улучшение **трансформерной модели** привело к **ухудшению качества** сегментации.
- Базовые версии обеих моделей имели схожее качество (примерно **0.25** IoU).

>  — улучшение метрики по сравнению с дефолтной  
>  — снижение метрики по сравнению с дефолтной


# Собственная имплементация

In [87]:
import segmentation_models_pytorch as smp
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.metrics import jaccard_score
from tqdm import tqdm

# Загрузка своих датасетов (train_loader, val_loader) должна быть определена заранее

# Имплементация 2-х моделей:
models = {
    "Unet_resnet34": smp.Unet(encoder_name="resnet34", encoder_weights=None, in_channels=3, classes=1),
    "Unet_mobilenet": smp.Unet(encoder_name="mobilenet_v2", encoder_weights=None, in_channels=3, classes=1)
}


In [78]:
def train_model(model, train_loader, epochs=3, lr=1e-4):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = smp.losses.DiceLoss(mode='binary')
    model.train()
    for epoch in range(epochs):
        for images, masks in tqdm(train_loader):
            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, masks)
            loss.backward()
            optimizer.step()
    return model


In [79]:
def evaluate_model(model, val_loader):
    model.eval()
    preds, gts = [], []
    with torch.no_grad():
        for images, masks in val_loader:
            images = images.to(device)
            outputs = model(images)
            preds.append((outputs > 0.5).cpu().numpy().astype(int))
            gts.append(masks.cpu().numpy().astype(int))
    # Flatten
    preds = np.concatenate(preds).reshape(-1)
    gts = np.concatenate(gts).reshape(-1)
    iou = jaccard_score(gts, preds)
    return iou


In [82]:
from torchvision import transforms

val_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor()
])


In [110]:
import glob

paths = glob.glob('/content/smokedata/Validation/Validation/*.jpg')
print(f"Найдено {len(paths)} изображений")

Найдено 180 изображений


In [111]:
from torch.utils.data import DataLoader

val_dataset = SmokeDataset('/content/smokedata/Validation/Validation', transform=val_transform)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)


180


In [112]:
baseline_iou_scores = {
    "Unet_resnet34_pretrained": 0.68,
    "Unet_mit_b0": 0.70
}

trained_scores = {}
for name, model in models.items():
    print(f"\nTraining {name}...")
    model = train_model(model, train_loader)
    iou = evaluate_model(model, val_loader)
    trained_scores[name] = iou



Training Unet_resnet34...


100%|██████████| 179/179 [00:12<00:00, 14.54it/s]



Training Unet_mobilenet...


100%|██████████| 179/179 [00:09<00:00, 17.91it/s]


In [113]:
for name in trained_scores:
    print(f"{name} IoU: {trained_scores[name]:.4f}")
    print(f"Сравнение с бейзлайном: Δ={trained_scores[name] - baseline_iou_scores.get(name+'_pretrained', 0):.4f}")


Unet_resnet34 IoU: 0.4833
Сравнение с бейзлайном: Δ=-0.1967
Unet_mobilenet IoU: 0.4500
Сравнение с бейзлайном: Δ=0.4500
